In [2]:
import serial
import time

SERIAL_PORT = "/dev/ttyUSB0"
# after writing commands a pause is needed
CMD_WRITE_PAUSE = 0.2

MM12_READ_INFOS   = b'\x55\x55\x00\x00\xaa'
MM12_READ_DISPLAY = b'\x55\x55\x01\x00\xab'

try:
    serial = serial.Serial(port=SERIAL_PORT, baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=2)

except serial.SerialException:
    print("Could not open serial port '{0}'!".format(SERIAL_PORT))
    raise

In [3]:
# function codes of Benning MM12 (table 6 in communication datasheet)
# the codes 0x32 ... 0x3E have not been included at the moment, because they are rarely used
list_function_codes = {
    0x00 : 'None',
    0x01 : 'AC V',
    0x02 : 'DC V',
    0x03 : 'AC mV',
    0x04 : 'DC mV',
    0x05 : 'Ohm',
    0x06 : 'Continuity',
    0x07 : 'Diode',
    0x08 : 'Capacitor',
    0x09 : 'AC A',
    0x0A : 'DC A',
    0x0B : 'AC mA',
    0x0C : 'DC mA',
    0x0D : '°C',
    0x0E : '°F',
    0x0F : 'Frequency',
    0x10 : 'Duty',
    0x11 : 'Hz (V)',
    0x12 : 'Hz (mV)',
    0x13 : 'Hz (A)',
    0x14 : 'Hz (mA)',
    0x15 : 'AC+DC (V)',
    0x16 : 'AC+DC (mV)',
    0x17 : 'AC+DC (A)',
    0x18 : 'AC+DC (mA)',
    0x19 : 'LPF (V)',
    0x1A : 'LPF (mV)',
    0x1B : 'LPF (A)',
    0x1C : 'LPF (mA)',
    0x1D : 'AC uA',
    0x1E : 'DC uA',
    0x1F : 'DC A out',
    0x20 : 'DC A out (Slow Linear)',
    0x21 : 'DC A out (Fast Linear)',
    0x22 : 'DC A out (Slow Step)',
    0x23 : 'DC A out (Fast Step)',
    0x24 : 'Loop Power',
    0x25 : '250 Ohm HART',
    0x26 : 'Voltage Sense',
    0x27 : 'Peak Hold (V)',
    0x28 : 'Peak Hold (mV)',
    0x29 : 'Peak Hold (A)',
    0x2A : 'Peak Hold (mA)',
    0x2B : 'LoZ AC V',
    0x2C : 'LoZ DC V',
    0x2D : 'LoZ AC+DC (V)',
    0x2E : 'LoZ LPF (V)',
    0x2F : 'LoZ Hz (V)',
    0x30 : 'LoZ Peak Hold (V)',
    0x31 : 'Battery',
    
    0xAC : 'LoZ Auto Sense' # it's 0x2C, but in Auto Sense mode (Bit 7 is set)
}

#list_function_codes

In [193]:
# range codes of Benning MM12 (table 7.1 and 7.2 in communication datasheet)
list_range_multiplier_ohm = {
    0x00 : 0.01,
    0x01 : 0.1,
    0x02 : 1,
    0x03 : 10,
    0x04 : 100,
    0x05 : 1000
}

list_range_multiplier_temp = {
    0x00 : 0.1
}

list_range_multiplier_voltage = {
    0x00 : 0.0001,
    0x01 : 0.001,
    0x02 : 0.01,
    0x03 : 0.1
}

In [237]:
def func_mask_autoRangeBit(byte_in):
    byte_out = byte_in & 0b01111111
    return byte_out

In [250]:
#message_bytes = bytes.fromhex(MM12_READ_INFOS)

#MM12_READ_INFOS   = b'\x55\x55\x00\x00\xaa'
print(MM12_READ_DISPLAY)

serial.write(MM12_READ_DISPLAY)
#time.sleep(CMD_WRITE_PAUSE)

b'UU\x01\x00\xab'


5

In [251]:
hex_string = serial.read(17).hex()

# fill string with spaces for better reading of the hex string
hex_string_wSpaces = " ".join(hex_string[i-1:i+1] for i, c in enumerate(hex_string) if i%2)

print(hex_string_wSpaces)

55 55 01 0c 0e 00 e2 02 00 99 00 00 00 70 00 00 b2


In [252]:
# byte 0 from data payload is the function code
func_code = hex_string[8:10]
# convert from hex to int
func_code_int = int(func_code, 16)
func_str = list_function_codes[func_code_int]

print(func_str)

°F


In [253]:
# byte 1 from data payload is the range code
range_code = hex_string[10:12]
# convert from hex to int
range_code_int = int(range_code, 16)

# mask the MSB that identifies the auto or manual range
range_code_int = func_mask_autoRangeBit(range_code_int)

# function: Ohm
if func_code_int == 0x05:
    range_int = list_range_multiplier_ohm[range_code_int]
# function: Temp (°C or °F)
elif ( func_code_int == 0x0D or 
        func_code_int == 0x0E ):
    range_int = list_range_multiplier_temp[range_code_int]
# function: Voltage (AC V, LPF (V), Peak Hold (V), DC V, AC+DC (V))
elif ( func_code_int == 0x01 or
        func_code_int == 0x19 or
        func_code_int == 0x27 or
        func_code_int == 0x02 or
        func_code_int == 0x15 ):
    range_int = list_range_multiplier_voltage[range_code_int]
    
print(range_int)

0.1


In [254]:
disp_value_hex = bytes.fromhex(hex_string[12:18])
#disp_value_hex

disp_value_sint = int.from_bytes(disp_value_hex, 'little', signed=True)
disp_value_sint

738

In [255]:
print("Display value: {} {}".format(disp_value_sint*range_int, func_str))

Display value: 73.8 °F
